# Exercise training a sklearn model on Cloud AI Platform

Before you can start training the model we have to make sure that we have the required dependency installed. 

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
!pip install pandas-gbq

### Local training
First we have to see if our model trains locally. This will help us debug any issues before training on the Google Cloud Platform. Please make sure that you set the --pathoutput. 

In [1]:
!gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=trainer/ \
   -- \
   --pathdata gs://erwinh-mldemo/scikit/marketing-data.csv \
   --pathoutput <your_bucket_path> \ # set to your bucket. Example: gs://bucket/folder 
   --storage BQ \
   --bqtable kfp-primer-workshop.marketing_data.raw

[True, False, False, False, True, False, False, False, True, False, True, True, True, True, False]
[False, True, True, True, False, False, False, True, False, True, False, False, False, False, True]


model score: 0.904
pipeline run done :)


change job name: marketing_v1_98

In [23]:
!gcloud ai-platform jobs submit training marketing_v1_99 \
   --staging-bucket=gs://kfp-scikit \
   --region=us-central1 \
   --module-name=trainer.task \
   --package-path=trainer \
   --runtime-version 1.14 \
   --python-version 3.5 \
   -- \
   --pathdata gs://kfp-scikit/data/scikit/marketing-data.csv \
   --pathoutput gs://kfp-scikit/model/output \
   --storage BQ \
   --bqtable kfp-primer-workshop.marketing_data.raw

Job [marketing_v1_99] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe marketing_v1_99

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs marketing_v1_99
jobId: marketing_v1_99
state: QUEUED


In [28]:
%%bash
MODEL_NAME="Marketing"
MODEL_VERSION="v2"
MODEL_LOCATION='gs://kfp-scikit/model/output/model/' 

In [41]:
!gcloud ai-platform models create marketingpredictor

Created ml engine model [projects/kfp-primer-workshop/models/marketingpredictor].


In [44]:
!gcloud ai-platform versions create marketing_v1 \
--model marketingpredictor \
--origin gs://kfp-scikit/model/output/model/ \
--runtime-version 1.14 \
--framework scikit-learn \
--python-version 3.5

Creating version (this might take a few minutes)......done.                    


In [46]:
!gcloud ai-platform predict \
  --model marketingpredictor \
  --version marketing_v1 \
  --json-instances trainer/predict.json

[True, True]
